# 04 - Feature Engineering

## Overview
Build stint_features.parquet with all required columns for modeling.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
from f1ts import config, io_flat, features, validation

## Load

In [ ]:
processed_dir = config.paths()['data_processed']
raw_dir = config.paths()['data_raw']

laps_processed = io_flat.read_parquet(processed_dir / 'laps_processed.parquet')
sessions = io_flat.read_csv(raw_dir / 'sessions.csv')

pitloss_csv = str(config.paths()['data_lookups'] / 'pitloss_by_circuit.csv')
hazard_csv = str(config.paths()['data_lookups'] / 'hazard_priors.csv')

## Transform: Assemble Features

In [ ]:
stint_features = features.assemble_feature_table(
    laps_processed, sessions, pitloss_csv, hazard_csv
)

## Validate

In [ ]:
validation.validate_stint_features(stint_features)
print(f'\n✓ Feature table has {len(stint_features.columns)} columns')

## Save

In [ ]:
features_dir = config.paths()['data_features']
io_flat.write_parquet(stint_features, features_dir / 'stint_features.parquet')

# Also save degradation training subset
deg_train = stint_features[stint_features['target_deg_ms'].notna()].copy()
io_flat.write_parquet(deg_train, features_dir / 'degradation_train.parquet')

print('✓ Saved feature tables')

## Repro Notes

- Added rolling pace features
- Estimated degradation slopes
- Joined pit loss and hazard lookups
- Created degradation target variable